# Data Ingestion Pipeline Research

In [12]:
import os

# check the current working dir
%pwd

'g:\\ML_DL_Projects\\MLOPS_Indian_Flight_Price_Prediction\\research'

In [13]:
# changing the dir to main repo
os.chdir("../")

%pwd

'g:\\ML_DL_Projects\\MLOPS_Indian_Flight_Price_Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path

# step 1 in workflow for Data ingestion
@dataclass
class DataIngestionConfig:
    root_dir : Path
    # source_URL : str
    kaggle_dataset : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
# step 2 and 3 not needed now
# step 4 later on
# step 5 in workflow for data ingestion

from src.indian_flight_price_prediction.constants import *
from src.indian_flight_price_prediction.utils.common import read_yaml, create_directories


In [17]:
class ConfigurationManager:
    def __init__(self,
                config_file_path = CONFIG_FILE_PATH,
                params_file_path = PARAMS_FILE_PATH,
                schema_file_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories(config.root_dir)

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_dataset = config.kaggle_dataset,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
from src.indian_flight_price_prediction.logger import logger
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile

In [8]:
# compnent - Data Ingestion 

class DataIngestion:
    def __init__(self, config : DataIngestionConfig ):
        self.config = config
        self.api = KaggleApi()
        self.api.authenticate()
        
        # to be added to the code 
        # try:
        #     self.api.authenticate()
        # except Exception as e:
        #     raise RuntimeError("Failed to authenticate with Kaggle API") from e
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            # filename, headers = request.urlretrieve(
            #     url = self.config.source_URL,
            #     filename = self.config.local_data_file
            # )
            dataset_name = self.config.kaggle_dataset
            logger.info(f"Dataset {dataset_name} downloading!!")
            self.api.dataset_download_files(
                dataset = dataset_name,
                path = os.path.dirname(self.config.local_data_file),
                force = True,
                quiet = False 
            )
            logger.info(f"Dataset {dataset_name} downloaded at {self.config.local_data_file}!!")
        else:
            logger.info(f"Dataset file already exists locally.")

    # extract zip file
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        zip_file_path = self.config.local_data_file
        if not zip_file_path.endswith(".zip"):
            zip_file_path += ".zip"

        with zipfile.ZipFile(zip_file_path, 'r' ) as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Data extracted to {unzip_path}")

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-20 18:20:49,230 : INFO : common : yaml file : config\config.yaml loaded successfully.]
[2025-04-20 18:20:49,232 : INFO : common : yaml file : params.yaml loaded successfully.]
[2025-04-20 18:20:49,235 : INFO : common : yaml file : schema.yaml loaded successfully.]


AttributeError: module 'os' has no attribute 'makedir'